## Load RCall
- run R instance in the background
- establish connection between Julia and R
- uses the R_HOME or default R location in certain OS to run R binary


In [ ]:
using RCall

## R"" string macro
- easiest way to interface with R
- exact R statements
- returns an R object which can be converted to Julia using rcopy

In [ ]:
aq_r=R"airquality" # get an R object dataframe

In [ ]:
rtoj=rcopy
aq_j = rtoj(aq_r); first(aq_j,3)  # convert to Julia dataframe

In [ ]:
using DataVoyager
aq_j |> Voyager

In [ ]:
using DataVoyager

## Let's load ggplot2 and plot airquality

In [ ]:
R"library(ggplot2)"

In [ ]:
p1=R"ggplot(data=airquality)+geom_point(aes(x=Wind,y=Solar.R,color=Temp))"

In [ ]:
p1=R"ggplot(data=airquality)+geom_point(aes(x=Wind,y=Ozone,color=Temp))+facet_grid(Month ~ .)"

In [ ]:
rcopy(p1) # translate robj to julia which is a dictionary type

In [ ]:
p1=R"ggplot(data=airquality)+geom_point(aes(x=Ozone,y=Solar.R,color=Temp))"

In [ ]:
R"plot(airquality)"

In [ ]:
R"library(randomForest)"

In [ ]:
R"rfmodel=randomForest(Temp ~ .,data=airquality,na.action=na.omit)" # regression

In [ ]:
R"varImpPlot(rfmodel)"

In [ ]:
R"rfmodel"

In [ ]:
R"randomForest(as.factor(Month) ~ .,data=airquality,na.action=na.omit)" # classification

In [ ]:
R"library(caret)"

In [ ]:
R"crf=train(Temp ~ .,data=airquality,method='rf',na.action=na.omit)"

In [ ]:
R"ctreebag=train(Temp ~ .,data=airquality,method='treebag',na.action=na.omit)"

In [ ]:
crf_j=rcopy(@rget crf);
ctreebag_j=rcopy(@rget ctreebag)

## Julia wrappers of R caret using RCall APIs

In [ ]:
function fit!(learner,x,y)
    xx = x |> DataFrame
    yy = y |> Vector
    rres = rcall(:train,xx,yy,method=learner)
    #rres = R"train($xx,$yy,method=$learner)"
    rres
end

function transform!(model,x)
    xx = x |> DataFrame
    res = rcall(:predict,model,xx) #in robj
    #res=R"predict($model,$x)"
    return rcopy(res) # return extracted robj
end


In [ ]:
using DataFrames
learner = "rf"
iris=rcopy(R"iris")
x=iris[:,1:4] 
y=iris[:,5] 
model = fit!(learner,x,y) #robject
print(model)
prediction=transform!(model,x) |> collect;

In [ ]:
R"table($prediction,$y)"

## Grid search for parameter optimization

In [ ]:
R"tunegrid <- expand.grid(.mtry=c(1:10))"

In [ ]:
R"control <- trainControl(method='repeatedcv', number=3, repeats=3, search='grid')"

In [ ]:
R"crf=train(Temp ~ .,data=airquality,method='rf',na.action=na.omit,tuneGrid=tunegrid, trControl=control)"

In [ ]:
R"plot(crf)"

In [ ]:
R"dcomp=airquality[complete.cases(airquality),]"
R"bestmtry <- tuneRF(dcomp[,-4],dcomp[,4], ntree=500)"

In [ ]:
iris_j=rcopy(R"iris")

In [ ]:
model=R"train(Species ~ .,data=$iris_j,method='rf')"

In [ ]:
rcopy(model)